In [1]:
import pandas as pd
from elasticsearch import Elasticsearch

In [2]:
df = pd.read_csv('data/corpus.csv')

In [3]:
df.head()

,Poetic Opening,Poem,Author,Lines,Count of Metaphor,Metaphorical Name,meaning,year,extra1,meaning1
0,தற்கொலை புரியப்போய் மரணம் மறுதலிக்க,கூப்பிடு தூரத்தில் வாழ்க்கை,வைரமுத்து,அதோ பார் உயிரில் பாதி ஒழுகிவிட்டாலும் மிச்ச உய...,1,அங்குலப் புழு,நீளம் சிறுத்த புளு,2020,NaN,NaN
1,தற்கொலை புரியப்போய் மரணம் மறுதலிக்க,கூப்பிடு தூரத்தில் வாழ்க்கை,வைரமுத்து,பறவையாய் இருந்தும் பறக்காத கோழி பருந்து கண்டது...,1,மரணம் என்ற நிஜத்திற்கு,மரணம் என்பது தவிர்க்க முடியாதது,2020,NaN,NaN
2,தற்கொலை புரியப்போய் மரணம் மறுதலிக்க,கூப்பிடு தூரத்தில் வாழ்க்கை,வைரமுத்து,வலையறுந்து நிலை குலைந்தும் அந்த ஓரிழைச் சிலந்த...,1,ஓரிழைச் சிலந்தி,தனிமைப்படுத்தப்பட்ட நிலையில் இருப்பவர்,2020,NaN,NaN
3,அதிகாலை ஒலிகள் ஐந்துமணிப் பறவைகள்,இது போதும் எனக்கு,வைரமுத்து,அதிகாலை ஒலிகள் ஐந்துமணிப் பறவைகள்,1,ஐந்துமணிப் பறவைகள்,புதிய நாள் தொடங்கும் புதிய தொடக்கங்களை குறிக்க...,2011,NaN,NaN
4,அதிகாலை ஒலிகள் ஐந்துமணிப் பறவைகள்,இது போதும் எனக்கு,வைரமுத்து,இருட்கதவுதட்டும் சூரியவிரல் பள்ளியெழுச்சி பாடு...,2,இருட்கதவுதட்டும் சூரியவிரல்,இருளில் இருந்து வெளிச்சத்திற்கு செல்லும் பயணத்...,2011,பள்ளியெழுச்சி பாடும் உன் பாதக்கொலுசு,புதிய நாளில் புதிய வேலைகளைத் தொடங்குவதை குறிக்...


In [4]:
mapping = {
    "settings": {
        "index": {
            "number_of_shards": 1,
            "number_of_replicas": 1
        },
        "analysis": {
            "analyzer": {
                "tamil_ngram_analyzer": {
                    "type": "custom",
                    "tokenizer": "classic",
                    "filter": [
                        "ngram_filter"
                    ]
                },
                "tamil_ngram_analyzer_1": {
                    "type": "custom",
                    "tokenizer": "classic",
                    "filter": [
                        "ngram_filter_1"
                    ]
                },
                "tamil_ngram_analyzer_2": {
                    "type": "custom",
                    "tokenizer": "classic",
                    "filter": [
                        "ngram_filter_2"
                    ]
                },

                "tamil_analyzer_sw": {
                    "type": "custom",
                    "tokenizer": "classic",
                    "filter": [
                        "custom_stopword"
                    ]
                },
                "tamil_analyzer_st": {
                    "type": "custom",
                    "tokenizer": "classic",
                    "filter": [
                        "custom_stemmer"
                    ]
                },
                "tamil_analyzer_st_sw": {
                    "type": "custom",
                    "tokenizer": "classic",
                    "filter": [
                        "custom_stemmer",
                        "custom_stopword"
                    ]
                },
                "tamil_analyzer_sw_st": {
                    "type": "custom",
                    "tokenizer": "classic",
                    "filter": [
                        "custom_stopword",
                        "custom_stemmer"
                    ]
                },
                "tamil_analyzer_syn": {
                    "type": "custom",
                    "tokenizer": "classic",
                    "filter": [
                        "custom_synonym"
                    ]
                },
                "tamil_analyzer_syn_sw": {
                    "type": "custom",
                    "tokenizer": "classic",
                    "filter": [
                        "custom_synonym",
                        "custom_stopword"
                    ]
                },
                "tamil_analyzer_syn_st": {
                    "type": "custom",
                    "tokenizer": "classic",
                    "filter": [
                        "custom_synonym",
                        "custom_stemmer"
                    ]
                },
                "tamil_analyzer_syn_sw_st": {
                    "type": "custom",
                    "tokenizer": "classic",
                    "filter": [
                        "custom_synonym",
                        "custom_stopword",
                        "custom_stemmer"
                    ]
                }
            },
            "filter": {
                "ngram_filter": {
                    "type": "edge_ngram",
                    "min_gram": 2,
                    "max_gram": 20,
                    "side": "front"
                },
                "ngram_filter_1": {
                    "type": "edge_ngram",
                    "min_gram": 1,
                    "max_gram": 20,
                    "side": "front"
                },
                "ngram_filter_2": {
                    "type": "edge_ngram",
                    "min_gram": 4,
                    "max_gram": 20,
                    "side": "front"
                },
                "custom_stopword": {
                    "type": "stop",
                    "stopwords_path": "analyze/stopwords.txt"
                },
                "custom_stemmer": {
                    "type": "stemmer_override",
                    "rules_path": "analyze/stem.txt"
                },
                "custom_synonym": {
                    "type": "synonym",
                    "synonyms_path": "analyze/synonyms.txt"
                }

            }}},
    "mappings": {
        "properties": {
            "Poem": {
                "type": "text",
                "analyzer": "tamil_analyzer_syn_sw_st",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "Author": {
                "type": "text",
                "analyzer": "tamil_analyzer_syn_sw_st",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "Poetic Opening": {
                "type": "text",
                "analyzer": "tamil_analyzer_syn_sw_st",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "Lines": {
                "type": "text",
                "analyzer": "tamil_analyzer_syn_sw_st",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "Metaphorical Name": {
                "type": "text",
                "analyzer": "tamil_analyzer_syn_sw_st",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "meaning": {
                "type": "text",
                "analyzer": "tamil_analyzer_syn_sw_st",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "ignore_above": 256
                    }
                }
            },
            "year": {
                "type": "integer"
            },
            "Count of Metaphor":{
                "type":"integer"
            }
        }
    }
    
}

In [5]:
es = Elasticsearch("http://localhost:9200", verify_certs=False, basic_auth=['elastic', 'ImqurIpw-n7ZWFKH6Kvv'])
INDEX = 'songs_7'